In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [177]:
from Preprocessor import Preprocessor

#FilterReview
from sklearn.model_selection import train_test_split 
from sklearn.cluster import KMeans
from sklearn import metrics 
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import pickle 
import os.path
'''
readCSV :CSV 파일을 읽는다. 고정포맷이 정해짐 
trainModel(ts =0.25): 입력된 모델에서 학습시킨다. ts 테스트 케이스 
predict(): 학습시킨 모델에 test데이터가 입력되면 예측한다. 
writePredictionResult(): 예측 결과를 파일로 출력한다. 
'''
class FilterReviewUSV:
    def __init__(self):
        self.model_filename = 'model/FilterReviewUSV_model'
        self.isSetModel = False
        if not(self.isSetModel) and os.path.isfile(self.model_filename+'.sav'):
            self.kmeans_data = pd.read_csv(self.model_filename+'.csv',encoding='euc-kr')
            self.kmeans = pickle.load(open(self.model_filename+'.sav', 'rb'))
            
    def input_X(self, para):
        pre = Preprocessor()
        pre.setPara(para)
        self.X = pre.transVector()
    def input_Y(self, result):
        self.Y = result
        
    def readCSV(self,filename,tag=True):
        self.data = pd.read_csv(filename,encoding='euc-kr')
        self.X_content = self.data['글'].values
        self.input_X(self.X_content)
        if tag == True:
            self.input_Y(self.data['tag'].values)
        
    def trainModel(self):
        
        #self.X_train,self.X_test,self.y_train,self.y_test=train_test_split(self.X,self.Y,test_size=ts)
        #print(self.y_train,self.y_test)
        self.kmeans  = KMeans(n_clusters=2).fit(self.X) 

        #self.y_pred = self.kmeans.fit(self.X_train, self.y_train).predict(self.X_test)

        # 테스트 socre 확인
        #print("Number of right labeled points out of a total %d points : %.2f %%"
      #% (len(self.X),100*(len(self.X)-(self.y_test != self.y_pred).sum())/len(self.X)))
        
        self.data['category'] = self.kmeans.labels_
        self.kmeans_data = self.data
        
        self.kmeans_data.to_csv(self.model_filename+'.csv',encoding='euc-kr')
        pickle.dump(self.kmeans, open(self.model_filename+'.sav', 'wb'))
        
        self.isSetModel = True
    def predict(self):
        self.y_pred = self.kmeans.predict(self.X)
        self.y_pred
        print('\n\n\n')
        print(self.y_pred)
        tolabel = np.zeros(self.y_pred.shape[0],dtype='int32')
        print(tolabel)
        for i in range(self.y_pred.shape[0]):
            pred = self.y_pred[i]
            sample_set = self.kmeans_data[self.kmeans_data['category']==pred]
            #print(sample_set['category'].head())
            
            tolabel[i] = 1 if sample_set['tag'].mean()>0.5 else 0
            
        tolabel_pd = pd.DataFrame({'P_tag':tolabel})
        tolabel_pd.reset_index(inplace=True)
        self.data.reset_index(inplace=True)
        print(tolabel_pd)
        print(self.data)
        self.data_labeled = pd.merge(self.data,tolabel_pd,on='index' )
        self.data_labeled.drop(labels='index',axis=1,inplace=True)
        print(self.data_labeled.head())
        return self.data_labeled
    
    def writePredictionResult(self, filename):
        self.data_labeled.to_csv(filename,encoding='euc-kr')

In [180]:
fr = FilterReviewUSV()
#fr.input_X('나는 멍청이다.\n취업 좀 하자\n취업 좀 하자\n취업 좀 하자')
#fr.input_Y([1,0,1,0])
#fr.readCSV(tag=True,filename='Train.csv')
#fr.trainModel()

fr.readCSV(tag=False,filename='Test.csv')
fr.predict()

fr.writePredictionResult('sample.csv')
#print(fr.y_pred)
#fr.X_content


['나는 멍청하다.' '지수는 왜 나갔을까?' '언제 취업하지…' '잠온다.']




[0 1 1 1]
[0 0 0 0]
   index  P_tag
0      0      1
1      1      0
2      2      0
3      3      0
   index            글    조회수
0      0     나는 멍청하다.  10000
1      1  지수는 왜 나갔을까?    200
2      2     언제 취업하지…    300
3      3         잠온다.      2
             글    조회수  P_tag
0     나는 멍청하다.  10000      1
1  지수는 왜 나갔을까?    200      0
2     언제 취업하지…    300      0
3         잠온다.      2      0
